## Le code de chargement du fichier.

In [8]:
# compatibilité python 2 et python 3
from __future__ import division, print_function, unicode_literals

# imports
import numpy as np
import os

# stabilité du notebook d'une exécution à l'autre
np.random.seed(42)

# jolies figures directement dans le notebook
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# ignorer les warnings inutiles (voir SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [38]:
# Get the dataset

import pandas as pd
import pathlib
import imageio

#Blurry images
paths_blurry = pathlib.Path('./dataset/Livrable1/Blurry/').glob('*.jpg') #Get all files blurred
sorted_blurry = sorted([x for x in paths_blurry]) #Order by number

paths_noisy = pathlib.Path('./dataset/Livrable1/Noisy/').glob('*.jpg') #Get all files noised
sorted_noisy = sorted([x for x in paths_noisy]) #Order by number

#test
print("Blurry path 01 :",sorted_blurry[0]) #For instance, return the first blurry image path
print("Noizy path 04", sorted_noisy[3]) #For instance, return the 4th noisy image path

#im = imageio.imread(str(im_path))


Blurry path 01 : dataset\Livrable1\Blurry\blurry_001.jpg
Noizy path 04 dataset\Livrable1\Noisy\noisy_004.jpg


## Le code du débruitage sur un sous-ensemble d’images bruitées.

In [ ]:
#code commenté

explication

## Le code de l’affutage sur un sous-ensembles d’images floutées.

In [ ]:
#code commenté

explication du code

## Une étude de cas explicitant les compromis entre ces deux opérations. Cette partie du livrable doit inclure le bruitage d’images et montrer la perte de détails, ou l’affutage d’images et montrer l’apparition du bruit.

### Bruitage d'images

In [ ]:
#code commenté

### Affutage d'images

In [ ]:
#code commenté

### Explication des compromis

In [ ]:
#code commenté